# Collect articles about hurricane Helene coverage

1. Use wayback Machine to scrape everything from a news website

2. Use UNC access's News Bank

## 2. News Bank pdf about "Helene" coverage

Use Selenium to download news article pdfs from news bank

keyword: helene

date: Sep 2024 (when the first 3 articles about Helene were published) - Oct 2024?

Location: USA - North Carolina

In [ ]:
# Import Module
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Scrape multiple pages

In [ ]:
# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

# Open URL
url = "https://infoweb.newsbank.com/apps/news/results?hide_duplicates=2&offset=0&maxresults=60&sort=YMD_date%3AD&p=AMNEWS&t=country%3AUSA%21USA/state%3ANC%21USA%2B-%2BNorth%2BCarolina&f=advanced&val-base-0=helene&fld-base-0=alltext&bln-base-1=and&val-base-1=Sep%202024%20-%20Oct%202024&fld-base-1=YMD_date"
driver.get(url)
time.sleep(5)
# Download pdf
try:
    # For I pages
    for i in range(2):
        # Click "select all" box
        # await driver.find_element(By.CLASS_NAME, 'search-hits__select-all form-checkbox').click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "search-hits__select-all.form-checkbox"))).click()
        # Click next
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "pager__item.pager__item--next"))).click()
        time.sleep(1)
except Exception as e:
    print(f"Cannot select page {i}")
    print(e)
else:
    # Click download button
    driver.find_element(By.CLASS_NAME, 'actions-bar__button.actions-bar__button--download').click()
    time.sleep(1)
    # While there is "Next" button
    while True:
        # Click "Download" button in pop-up
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "u-button multidoc-download__button"))).click()
        # wait loader to appear and then disappear
        WebDriverWait(driver, 10).until(EC.element_to_be_selected((By.CLASS_NAME, 'multidoc__button-spinner')))
        WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.CLASS_NAME, 'multidoc__button-spinner')))
        # Click "Next"
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "multidocs_pager_nav next"))).click()
        except:
            print("No Next found")
            break

driver.quit()





There is security check of robot. Can't pass ?

Ended up with manually downloading 5000+ articles by groups

In [1]:
import pdfplumber
import os
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re


extract text from pdf with <a href = 'https://github.com/jsvine/pdfplumber'>pdfplumber</a>

In [3]:
#   FUNC extract_text_from_pdf()
#   Input: directory (str) of pdf
#   Output: header (list) - article header information
#         article (list) - article text
def extract_text_from_pdf(dir, headers, articles):
    # header, article = [],[]
    with pdfplumber.open(dir) as pdf:
        for page in pdf.pages:
            # extract texts on each page and split texts by "OpenURL Link"
            texts = page.extract_text().split("\nOpenURL Link\n")
            # there is no "OpenURL Link" on that page
            if len(texts) == 1:
                articles[-1] += texts[0]
            else:
                headers.append(texts[0])
                articles.append(texts[1])
    return 

In [7]:
#   FUNC: decompose_header
#   Input: header (str)
#   Output: append titles, dates, newspapers, authors, and word_counts
def decompose_header(header):


    date_match = re.search(r"\b(September|October) \d{1,2}, \d{4}\b", header)
    
    date = date_match.group(0) if date_match else ""
    dates.append(date)
    loc = header.find(date)
    titles.append(header[:loc].replace("\n", ""))

    texts = header[loc:].split('\n')
    newspapers.append(texts[0][len(date)+1:].replace('| ', ""))

    # Line three
    match = re.search(r"Author: (.*?)Section: ", texts[1])
    author = match.group(1) if match else ""
    if author == "":
        author = texts[1][8:]
    authors.append(author)
    match_word = re.search(r"(\d+)\s*Words", texts[1])
    word_count = match_word.group(1) if match_word else ""
    word_counts.append(word_count)

    return

In [9]:
# FUNC: correct_news_name
# Input: newspaper name from NewsBank(str) e.g. Watauga Democrat, The (Boone, NC)
# Output: newspaper name corrected e.g. The Watauga Democrat
def correct_news_name(newspaper):
    if ", The" in newspaper:
        loc = newspaper.find(", The")
        newspaper = newspaper[:loc]
    elif "(" in newspaper:
        loc = newspaper.find('(')
        newspaper = newspaper[:loc]

    # Get rid of "The" at the beginning
    pattern = r"^The\s"
    newspaper = re.sub(pattern, "", newspaper)
    
    return newspaper

In [ ]:
headers1, articles1 = [],[]

path = 'News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1 1.pdf'
with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            # extract texts on each page and split texts by "OpenURL Link"
            texts = page.extract_text()

In [ ]:
# Get all files under news_bank_pdf
directory = 'news_bank_pdf'
pdf_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".pdf")]

headers, articles = [],[]
# Extract articles and headers from pdfs
for path in pdf_paths:
    extract_text_from_pdf(path, headers, articles)

# Save articles and headers to dataframe
temp = pd.DataFrame({"header":headers,
                    "article": articles})

# temp.to_csv('articles_20241112.csv', index=False)

In [ ]:
# Decompose headers
titles, dates, newspapers, authors, word_counts = [],[],[],[],[]
temp['header'].apply(lambda x: decompose_header(x))

temp['title'] = titles
temp['date'] = dates
temp['newspaper'] = newspapers
temp['author'] = authors
temp['word_count'] = word_counts
temp['newspaper'] = temp['newspaper'].apply(lambda x: correct_news_name(x))

temp.head()

# temp.to_csv("articles_20241112_cleaned.csv", index=False)

### Extract text from newspaper layout with Tesseract

Install <a href='https://github.com/tesseract-ocr/tesseract'>Tesseract</a> through homebrew

In [ ]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image

In [ ]:
# Convert PDF pages to images
pages = convert_from_path("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf")

for page in pages:
    # Crop or split image into columns if necessary
    text = pytesseract.image_to_string(page, config='--psm 6')  # PSM 6 for uniform block detection
    print(text)


In [ ]:
import layoutparser as lp
from layoutparser.models.detectron2 import catalog
import copy
import os
import requests as requests
import layoutparser as lp
from pdf2image import convert_from_path

def load_model(
        config_path: str = 'lp://<dataset_name>/<model_name>/config',
):

    config_path_split = config_path.split('/')
    dataset_name = config_path_split[-3]
    model_name = config_path_split[-2]

    # get the URLs from the MODEL_CATALOG and the CONFIG_CATALOG 
    # (global variables .../layoutparser/models/detectron2/catalog.py)
    model_url = catalog.MODEL_CATALOG[dataset_name][model_name]
    config_url = catalog.CONFIG_CATALOG[dataset_name][model_name]

    # override folder destination:
    if 'model' not in os.listdir():
        os.mkdir('model')

    config_file_path, model_file_path = None, None

    for url in [model_url, config_url]:
        filename = url.split('/')[-1].split('?')[0]
        save_to_path = f"model/" + filename
        if 'config' in filename:
            config_file_path = copy.deepcopy(save_to_path)
        if 'model_final' in filename:
            model_file_path = copy.deepcopy(save_to_path)

        # skip if file exist in path
        if filename in os.listdir("model"):
            continue
        # Download file from URL
        r = requests.get(url, stream=True, headers={'user-agent': 'Wget/1.16 (linux-gnu)'})

        with open(save_to_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=4096):
                if chunk:
                    f.write(chunk)

    # load the label map
    label_map = catalog.LABEL_MAP_CATALOG[dataset_name]

    return lp.models.Detectron2LayoutModel(
        config_path=config_file_path,
        model_path=model_file_path,
        label_map=label_map
    )


In [ ]:
# Load the image of the PDF page (or convert PDF to image as above)
# image = Image.open("page_image.png")
pages = convert_from_path("News_Article__Fayetteville_Observer_The_NC___October_8_2024__p1.pdf")

# Error occurs: having trouble finding the model
# layout = lp.models.Detectron2LayoutModel("lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config")
# layout = lp.models.Detectron2LayoutModel("lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config")
# layout = lp.Detectron2LayoutModel(
# "./config.yaml",
# "./model_final.pth",
# extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
# label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
# )

# This works
# layout = load_model('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config')


detected_layout = layout.detect(pages[0])

for block in detected_layout:
    segment = block.extract(pages)
    text = pytesseract.image_to_string(segment)  # OCR on each segment
    print(text)

## Extract Texts from New Readable PDFs

### Extract text from newspaper layout with LayoutParser

In "helen_articles_readable"

In [ ]:
import pdfplumber

path = 'helene_articles_readable/News_Article__Charlotte_Observer_The_NC___October_2_2024__p2.pdf'
with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            # extract texts on each page and split texts by "OpenURL Link"
            texts = page.extract_text()

Charlotte Observer, The (NC) - October 2, 2024 - page 2
October 2, 2024 | Charlotte Observer, The (NC) | Charlotte, North Carolina | Page 2
2A THEC HARLOTOTEB SERVER WEDNESDOACYT OBE2R 2 024
1't
YOUR 7-DAY FORECAST AccuWeather Getn otificationso f nearbyl ightning! YOUR ACTIVITIES FORECAST
Get the AccuWeather App
Today Tonight Thursday Friday Saturday Sunday Monday Tuesday TodayT hu. Fri. Sat. Sun. Mon. Tue.
. " \ .I' .,. - ' '\,~iJ, ' . " \.1-(, - FISHINGIN DEX
' ' Q ' '
~--P
~ ~, ' ~
)',,,,
OUTDOOARC TIVITIEINS DEX
Mostlys unny Mainlyc lear, Somes unw ith Humidw ith a Partlys unny Partlys unny Partlys unny Plentyo f sun
9
andh umid warma ndh umid a showerh; umid briefs hower
HIKINGIN DEX
or two
g30
67° 80° 66° 80° 65° 83° 63° 80° 61° 77° 52° 71° 50°
-~
RealFee8l9, ' ReaFl eel6, 8' RealFee8lS " RealFee8l2, ' RealFee8l5 ' Realfee8l, 3' Realfee7l 9' Realfee7l 3' GOLFINIGN DEX
UVl nde" 5 UVl nde" 3 UVl nde" 2 UVl nde" 5 UVl nde" 5 UVl nde" 5 UVl nde" 5 6 ...-=--
The patented AccuWeather.

In [5]:
model = lp.models.Detectron2LayoutModel(
    config_path="lp://NewspaperNavigator/faster_rcnn_R_50_FPN_3x/config",
    label_map={0: "Photograph", 1: "Illustration", 2: "Map", 3: "Comics/Cartoon", 4: "Editorial Cartoon", 5: "Headline", 6: "Advertisement"}
)
print(model)


AssertionError: Checkpoint /Users/yanans/.torch/iopath_cache/s/6ewh6g8rqt2ev3a/model_final.pth not found!

In [7]:
import layoutparser as lp
from pdf2image import convert_from_path
import pytesseract
from PIL import Image

# Load a pre-trained layout detection model
# model = lp.models.Detectron2LayoutModel(
#     config_path="lp://NewspaperNavigator/faster_rcnn_R_50_FPN_3x/config",
#     label_map={0: "Photograph", 1: "Illustration", 2: "Map", 3: "Comics/Cartoon", 4: "Editorial Cartoon", 5: "Headline", 6: "Advertisement"}
# )

model = lp.Detectron2LayoutModel(
            config_path ='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config', # In model catalog
            label_map   ={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}, # In model`label_map`
            extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8] # Optional
)

# Convert PDF page to image
pages = convert_from_path(path)
page_image = pages[0]  # Example: Process the first page

# Detect layout elements
layout = model.detect(page_image)

# Visualize layout detection (optional)
lp.draw_box(page_image, layout, box_width=3, box_color="red").save("layout_detected.png")

# Perform OCR on each detected text block
for block in layout:
    if block.type == "Text":  # Check if block is a text region
        # Crop text region
        cropped_image = page_image.crop(block.coordinates)
        
        # Apply OCR
        text = pytesseract.image_to_string(cropped_image)
        print(f"Text in Block {block.id}:")
        print(text)


AssertionError: Checkpoint /Users/yanans/.torch/iopath_cache/s/dgy9c10wykk4lq4/model_final.pth not found!

In [11]:
help("detectron2")

Help on package detectron2:

NAME
    detectron2 - # Copyright (c) Facebook, Inc. and its affiliates.

PACKAGE CONTENTS
    _C
    checkpoint (package)
    config (package)
    data (package)
    engine (package)
    evaluation (package)
    export (package)
    layers (package)
    model_zoo (package)
    modeling (package)
    projects (package)
    solver (package)
    structures (package)
    tracking (package)
    utils (package)

VERSION
    0.6

FILE
    /Users/yanans/Desktop/helene_coverage/.venv/lib/python3.12/site-packages/detectron2/__init__.py




### Manually Extract Articles

Import to <a href="https://notebooklm.google/">NotebookLM</a>, an AI document assistant by Google

In [4]:
import pandas as pd

pdfs = pd.read_csv("pdfs.csv")
pdfs.head()

,header,article
0,"Images of Destruction - and Hope\nOctober 6, 2...","Hurricane Helene swept across the Southeast, c..."
1,They were in the basement frantically preparin...,They were in the basement frantically preparin...
2,Free legal assistance available for Helene sto...,As thousands of North Carolinians continue to ...
3,No power but only minor damage: Spruce Pine qu...,The world's main producer of high-purity quart...
4,Milton takes turn to target Florida on 'destru...,Orlando Sentinel/Tribune Content Agency \nORLA...


In [5]:
for_str = """Dane Jackson won't offer up any spoilers on his status for Sunday's game against the Atlanta Falcons.\nThe veteran cornerback, who has been on injured reserve since the season started, could make his Carolina Panthers debut this weekend. But for now, he's just doing what he's told, and not sharing those directions with anyone outside of Bank of America Stadium.\n"I'm just following the plan that they've got for me," Jackson said with a big smile on Thursday after practice.\nJackson signed a two-year deal with the team in free agency in March. He was projected to be the favorite at the No. 2 cornerback spot opposite Jaycee Horn, but he suffered a notable hamstring injury in training camp in August.\nAnd he has been sidelined ever since.\n"It's been a process, for sure," Jackson said. "Never had a (hamstring injury) to this extent, so it's definitely been a process. But I've been working with the strength staff, with the training room staff - doing my own thing on the side, too - just trying to get to it and get back as healthy as I can."\nJackson built a bond with teammates in trainers room\nDuring Jackson's stint on the sideline, he bonded with fellow veterans D.J. Wonnum and Amare Barno, who have been on the physically unable to perform (PUP) list since July.\nThe trio worked in the trainers room together as they went through their respective rehab assignments. The bond between Wonnum and Jackson, in particular, helped the pair get back on the right track to returning to the field.\n"We've definitely (grown) closer since we've both been hurt, we've both been out," Jackson said. "We both like to play around a lot. Getting each other through the day - sometimes, you come in here hurt, and you've got to find it yourself. Just getting each other through the days and bonding with each other and growing together as teammates for sure."\nJackson, who played four seasons with the Buffalo Bills, is eager to play. He signed with Carolina largely due to his relationship and background with GM Dan Morgan.\nThe GM bet on Jackson, who wants to make the most of his opportunity with his """
head = "Panthers CB Dane Jackson preparing to return from IR\nOctober 13, 2024 | Charlotte Observer, The (NC) | Charlotte, North Carolina | Page 36\nAuthor: Mike Kaye"
pdfs[pdfs['header'] ==head]['article'] += for_str

/var/folders/cn/fww9r0gd1mg20td4q8v12_lsknvlj7/T/ipykernel_29009/295378105.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pdfs[pdfs['header'] ==head]['article'] += for_str
/var/folders/cn/fww9r0gd1mg20td4q8v12_lsknvlj7/T/ipykernel_29009

In [10]:
# Basic cleaning
import re

titles, dates, newspapers, authors, word_counts = [],[],[],[],[]
pdfs['header'].apply(lambda x: decompose_header(x))

pdfs['title'] = titles
pdfs['date'] = dates
pdfs['newspaper'] = newspapers
pdfs['author'] = authors
pdfs['word_count'] = word_counts

pdfs['newspaper'] = pdfs['newspaper'].apply(lambda x: correct_news_name(x))

# pdfs.to_csv("new_pdfs_articles_20241209.csv", index=False)
pdfs.head()

,header,article,title,date,newspaper,author,word_count
0,"Images of Destruction - and Hope\nOctober 6, 2...","Hurricane Helene swept across the Southeast, c...",Images of Destruction - and Hope,"October 6, 2024",Charlotte Observer,THE CHARLOTTEOBSERVER,
1,They were in the basement frantically preparin...,They were in the basement frantically preparin...,They were in the basement frantically preparin...,"October 6, 2024",Charlotte Observer,MARTHA QUILLIN,
2,Free legal assistance available for Helene sto...,As thousands of North Carolinians continue to ...,Free legal assistance available for Helene sto...,"October 6, 2024",Charlotte Observer,CHYNA BLACKMON cblackmon@charlotteobserver.com,
3,No power but only minor damage: Spruce Pine qu...,The world's main producer of high-purity quart...,No power but only minor damage: Spruce Pine qu...,"October 6, 2024",Charlotte Observer,BRIAN GORDON,
4,Milton takes turn to target Florida on 'destru...,Orlando Sentinel/Tribune Content Agency \nORLA...,Milton takes turn to target Florida on 'destru...,"October 9, 2024",Charlotte Observer,RICHARD TRIBOU,


## Filter news article data

In [ ]:
# Concat with other articles
temp['newspaper'] = temp['newspaper'].apply(lambda x: correct_news_name(x))
temp = temp[temp['word_count'].isna() == False]

articles = pd.concat([temp, pdfs])

In [ ]:
county = pd.read_csv('helene_county_newsrooms/WNC Helene counties.csv')

# Clean the county name
county['County'] = county['County'].str.replace(' (County)', '')

news_census = pd.read_excel('helene_county_newsrooms/NC-News-Census-1.xlsx')

# Join two datasets
helene_news = pd.merge(county, news_census, how='left')

# Filter by 'newspaper' & 'digital' types only
helene_news.Type = helene_news.Type.str.lower()
helene_newspaper = helene_news[helene_news['Type'].isin(['newspaper','digital'])]

# Save it locally
# helene_newspaper.to_csv('helene_newspaper.csv', index=False)
# helene_newspaper = pd.read_csv('helene_newspaper.csv')

# Remove "The " at the start of the names
helene_newspaper['newspaper'] = helene_newspaper.Outlet.apply(lambda x: correct_news_name(x))

# Keep necessary info only
h_news = helene_newspaper[['County', 'Outlet','newspaper']]

# Merge
helene_articles = pd.merge(articles, h_news, how="inner")

# Save
# helene_articles.to_csv('helene_all_articles.csv', index=False)


/var/folders/cn/fww9r0gd1mg20td4q8v12_lsknvlj7/T/ipykernel_29009/603044345.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  helene_newspaper['newspaper'] = helene_newspaper.Outlet.apply(lambda x: correct_news_name(x))
